<a href="https://colab.research.google.com/github/pkwbim/ai-learning/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import tensorflow as tf
dataset = tf.keras.utils.get_file(
    fname = "aclImdb.tar.gz",
    origin= "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True 
)

In [27]:
dataset

'/root/.keras/datasets/aclImdb.tar.gz'

In [28]:
# 下載後的 keras 存的暫存路徑
!ls /root/.keras/datasets/aclImdb/train -ls

total 66616
20532 -rw-r--r-- 1 7297 1000 21021197 Apr 12  2011 labeledBow.feat
  360 drwxr-xr-x 2 7297 1000   368640 Apr 12  2011 neg
  332 drwxr-xr-x 2 7297 1000   339968 Apr 12  2011 pos
 1416 drwxr-xr-x 2 7297 1000  1449984 Apr 12  2011 unsup
40380 -rw-r--r-- 1 7297 1000 41348699 Apr 12  2011 unsupBow.feat
  600 -rw-r--r-- 1 7297 1000   612500 Apr 12  2011 urls_neg.txt
  600 -rw-r--r-- 1 7297 1000   612500 Apr 12  2011 urls_pos.txt
 2396 -rw-r--r-- 1 7297 1000  2450000 Apr 12  2011 urls_unsup.txt


In [29]:

import glob, os
import pandas as pd
def getdata(mid):
  dn = os.path.dirname(dataset)
  posfn = glob.glob(os.path.join(dn, "aclImdb", "train", "pos", "*"))
  negfn = glob.glob(os.path.join(dn, "aclImdb", "train", "neg", "*"))
  contents = []
  for fn in posfn + negfn:
    with open(fn, encoding="utf8") as f:
      contents.append(f.read())
  df = pd.DataFrame({
      "content":contents,
      "sentiment":[1] * len(posfn)  + [0] * len(negfn)
  })
  return df 



In [30]:
train_df= getdata("train")
test_df = getdata("test")

In [31]:
train_df

,content,sentiment
0,McConaughey in a horror/thriller? I had to see...,1
1,John Carpenter's Halloween<br /><br />Is it th...,1
2,Plot is never the strong point of a Fred Astai...,1
3,The House of the Spirits is a gripping tale of...,1
4,I loved this movie. It is a definite inspirati...,1
...,...,...
24995,"Between 1937 and 1939, Twentieth Century-Fox m...",0
24996,"Some might scoff, but there is actually a real...",0
24997,If you look at Corey Large's information here ...,0
24998,<br /><br />Worst. Movie. Ever.<br /><br />Wha...,0


In [32]:
# 把詞化成數字
NUM_WORDS = 3000
INPUT_DIM = NUM_WORDS + 1 # 3000(詞數) + 1（padding）
OUTPUT_DIM = 128
INPUT_LENGTH = 512

from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=NUM_WORDS) # 只挑詞頻高的 3000 字 
tok.fit_on_texts(train_df["content"])



In [33]:
# 順序是依詞頻決定的
# 0 別用掉，因它是拿來做 padding 的
tok.word_index # 雖然都列出來，但在使用時，只挑詞頻高的 n 個字，由宣告Tokenizer 時當參數帶入

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'he': 26,
 'be': 27,
 'one': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'who': 34,
 'so': 35,
 'from': 36,
 'like': 37,
 'her': 38,
 'or': 39,
 'just': 40,
 'about': 41,
 "it's": 42,
 'out': 43,
 'if': 44,
 'has': 45,
 'some': 46,
 'there': 47,
 'what': 48,
 'good': 49,
 'more': 50,
 'when': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'she': 56,
 'even': 57,
 'my': 58,
 'would': 59,
 'which': 60,
 'only': 61,
 'story': 62,
 'really': 63,
 'see': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'were': 68,
 'me': 69,
 'well': 70,
 'than': 71,
 'we': 72,
 'much': 73,
 'been': 74,
 'bad': 75,
 'get': 76,
 'will': 77,
 'do': 78,
 'also': 79,
 'into': 80,
 'people': 81,
 'other': 82,
 '

In [34]:
# sequences: 數字序列
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])
pd.DataFrame(x_train_seq)



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,8,3,186,705,10,66,5,64,11.0,10.0,13.0,767.0,7.0,7.0,1.0,111.0,6.0,576.0,8.0,2753.0,2.0,9.0,32.0,894.0,1276.0,2.0,651.0,220.0,4.0,286.0,167.0,140.0,3.0,52.0,1158.0,10.0,188.0,132.0,73.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,304,2186,7,7,6,9,1,830,186.0,19.0,4.0,29.0,55.0,54.0,276.0,21.0,5.0,313.0,18.0,5.0,69.0,9.0,6.0,2.0,207.0,77.0,27.0,1.0,19.0,6.0,2095.0,1257.0,2.0,77.0,207.0,1067.0,3.0,52.0,315.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,111,6,112,1,562,210,4,3,1794.0,2.0,17.0,18.0,790.0,1.0,878.0,6.0,1495.0,1794.0,2.0,128.0,213.0,122.0,29.0,205.0,33.0,294.0,2.0,1588.0,622.0,962.0,2.0,1618.0,1051.0,51.0,1.0,51.0,33.0,906.0,171.0,65.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,310,4,1,6,3,783,4,220.0,1220.0,295.0,2410.0,2.0,1160.0,1245.0,1718.0,488.0,2.0,718.0,664.0,5.0,110.0,16.0,29.0,91.0,1795.0,2.0,833.0,11.0,8.0,58.0,647.0,6.0,28.0,4.0,1.0,115.0,105.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,444,11,17,9,6,3,17,9.0,22.0,16.0,11.0,17.0,6.0,287.0,1.0,2353.0,39.0,287.0,2640.0,9.0,141.0,27.0,8.0,341.0,115.0,17.0,10.0,25.0,107.0,8.0,3.0,193.0,55.0,9.0,77.0,94.0,22.0,1165.0,85.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,197,2,1114,1675,90,3,4,440,105.0,187.0,947.0,1.0,127.0,4.0,1.0,198.0,9.0,13.0,574.0,12.0,1.0,1178.0,66.0,1.0,35.0,5.0,1125.0,11.0,12.0,3.0,245.0,120.0,45.0,2116.0,91.0,2471.0,2.0,1.0,8.0,2908.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,46,235,18,47,6,162,3,144,495.0,16.0,228.0,569.0,75.0,105.0,11.0,43.0,20.0,29.0,7.0,7.0,3.0,759.0,4.0,182.0,81.0,369.0,16.0,2.0,827.0,1953.0,137.0,5.0,19.0,538.0,2.0,127.0,53.0,617.0,80.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,44,22,165,30,1616,130,20,895,681.0,222.0,3.0,17.0,443.0,8.0,362.0,14.0,4.0,8.0,60.0,237.0,393.0,3.0,106.0,769.0,7.0,7.0,83.0,4.0,29.0,86.0,124.0,138.0,3.0,2135.0,17.0,123.0,3.0,752.0,2.0,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,7,7,246,17,123,7,7,48,13.0,1.0,1286.0,4.0,1421.0,11.0,1031.0,1209.0,36.0,1583.0,9.0,80.0,3.0,1550.0,784.0,15.0,1.0,1322.0,812.0,1143.0,12.0,518.0,10.0,479.0,1.0,164.0,112.0,293.0,1.0,201.0,348.0,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=INPUT_LENGTH)
x_test_pad = pad_sequences(x_train_seq, maxlen=INPUT_LENGTH)
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,8,3,186,705,10,66,5,64,11.0,10.0,13.0,767.0,7.0,7.0,1.0,111.0,6.0,576.0,8.0,2753.0,2.0,9.0,32.0,894.0,1276.0,2.0,651.0,220.0,4.0,286.0,167.0,140.0,3.0,52.0,1158.0,10.0,188.0,132.0,73.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,304,2186,7,7,6,9,1,830,186.0,19.0,4.0,29.0,55.0,54.0,276.0,21.0,5.0,313.0,18.0,5.0,69.0,9.0,6.0,2.0,207.0,77.0,27.0,1.0,19.0,6.0,2095.0,1257.0,2.0,77.0,207.0,1067.0,3.0,52.0,315.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,111,6,112,1,562,210,4,3,1794.0,2.0,17.0,18.0,790.0,1.0,878.0,6.0,1495.0,1794.0,2.0,128.0,213.0,122.0,29.0,205.0,33.0,294.0,2.0,1588.0,622.0,962.0,2.0,1618.0,1051.0,51.0,1.0,51.0,33.0,906.0,171.0,65.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,310,4,1,6,3,783,4,220.0,1220.0,295.0,2410.0,2.0,1160.0,1245.0,1718.0,488.0,2.0,718.0,664.0,5.0,110.0,16.0,29.0,91.0,1795.0,2.0,833.0,11.0,8.0,58.0,647.0,6.0,28.0,4.0,1.0,115.0,105.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,444,11,17,9,6,3,17,9.0,22.0,16.0,11.0,17.0,6.0,287.0,1.0,2353.0,39.0,287.0,2640.0,9.0,141.0,27.0,8.0,341.0,115.0,17.0,10.0,25.0,107.0,8.0,3.0,193.0,55.0,9.0,77.0,94.0,22.0,1165.0,85.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,197,2,1114,1675,90,3,4,440,105.0,187.0,947.0,1.0,127.0,4.0,1.0,198.0,9.0,13.0,574.0,12.0,1.0,1178.0,66.0,1.0,35.0,5.0,1125.0,11.0,12.0,3.0,245.0,120.0,45.0,2116.0,91.0,2471.0,2.0,1.0,8.0,2908.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,46,235,18,47,6,162,3,144,495.0,16.0,228.0,569.0,75.0,105.0,11.0,43.0,20.0,29.0,7.0,7.0,3.0,759.0,4.0,182.0,81.0,369.0,16.0,2.0,827.0,1953.0,137.0,5.0,19.0,538.0,2.0,127.0,53.0,617.0,80.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,44,22,165,30,1616,130,20,895,681.0,222.0,3.0,17.0,443.0,8.0,362.0,14.0,4.0,8.0,60.0,237.0,393.0,3.0,106.0,769.0,7.0,7.0,83.0,4.0,29.0,86.0,124.0,138.0,3.0,2135.0,17.0,123.0,3.0,752.0,2.0,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,7,7,246,17,123,7,7,48,13.0,1.0,1286.0,4.0,1421.0,11.0,1031.0,1209.0,36.0,1583.0,9.0,80.0,3.0,1550.0,784.0,15.0,1.0,1322.0,812.0,1143.0,12.0,518.0,10.0,479.0,1.0,164.0,112.0,293.0,1.0,201.0,348.0,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
import numpy as np
y_train = np.array(train_df["sentiment"])
y_test = np.array(test_df["sentiment"])


In [37]:
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
layers = [
          Embedding(INPUT_DIM, OUTPUT_DIM, mask_zero=True, input_length=INPUT_LENGTH),
          Flatten(),
          Dense(INPUT_LENGTH, activation="relu"),
          Dropout(0.25),
          Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 512, 128)          384128    
_________________________________________________________________
flatten_2 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               33554944  
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 1026      
Total params: 33,940,098
Trainable params: 33,940,098
Non-trainable params: 0
_________________________________________________________________


你會發現，embedding 的Param是 384128 = 3001 * 128


In [38]:

from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [39]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("embedding.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)       
]
model.fit(x_train_pad, 
     y_train, 
     batch_size=200,
     validation_split=0.1,
     epochs=50,
     verbose=2,
     callbacks=callbacks)

Epoch 1/50
113/113 - 10s - loss: 0.7652 - accuracy: 0.5567 - val_loss: 0.8579 - val_accuracy: 0.1440
Epoch 2/50
113/113 - 9s - loss: 0.3796 - accuracy: 0.8278 - val_loss: 0.4564 - val_accuracy: 0.8064
Epoch 3/50
113/113 - 8s - loss: 0.1857 - accuracy: 0.9321 - val_loss: 0.4643 - val_accuracy: 0.8184
Epoch 4/50
113/113 - 8s - loss: 0.0830 - accuracy: 0.9767 - val_loss: 0.4807 - val_accuracy: 0.8424
Epoch 5/50
113/113 - 8s - loss: 0.0259 - accuracy: 0.9952 - val_loss: 0.5177 - val_accuracy: 0.8488
Epoch 6/50
113/113 - 8s - loss: 0.0075 - accuracy: 0.9994 - val_loss: 0.7118 - val_accuracy: 0.8232
Epoch 7/50
113/113 - 8s - loss: 0.0027 - accuracy: 0.9999 - val_loss: 0.7895 - val_accuracy: 0.8232
